# Generates Rollout for existing Checkpoint

This notebook will demonstrate the clean usage of the `track_mjx.analysis.rollout` module, which allows user to load the checkpoint from the previous training run, and perform a rollout of the checkpoint. This module abstracted away all the boilerplate codes for initializing the environment, and it is very clean and digestible, while customizable.

### Step 1: Imports and recover the config

In [1]:
# set environment variables for rendering

import os
import sys
from pathlib import Path

main_path = Path().resolve().parent
if str(main_path) not in sys.path:
    sys.path.append(str(main_path))
    
%env MUJOCO_GL=egl
%env PYOPENGL_PLATFORM=egl

from omegaconf import OmegaConf
from track_mjx.analysis.rollout import (
    restore_config,
    create_rollout_generator,
    create_environment,
    create_inference_fn,
)
from jax import numpy as jp

# save and load
from track_mjx.analysis.utils import save_to_h5py, load_from_h5py
import h5py

# render
from track_mjx.analysis.render import render_from_saved_rollout, display_video


# recover the config
ckpt_path = "/root/vast/scott-yang/track-mjx/model_checkpoints/rodent_data/ReferenceClip.p_250127_062443"  # replace with your checkpoint path
config = restore_config(ckpt_path)
cfg = OmegaConf.create(config)

# make some changes to the config
# replace with absolute path to your data
# -- your notebook may not have access to the same relative path
cfg.data_path = "/root/vast/scott-yang/track-mjx/data/ReferenceClip.p"
cfg.train_setup.checkpoint_to_restore = ckpt_path

env: MUJOCO_GL=egl
env: PYOPENGL_PLATFORM=egl
latest checkpoint step: 149


### Step 2: Restore policy and make rollout functions

In [2]:
env = create_environment(cfg)
inference_fn = create_inference_fn(env, cfg)
generate_rollout = create_rollout_generator(cfg["reference_config"], env, inference_fn)

env._steps_for_cur_frame: 2.0
latest checkpoint step: 149


### Step 3: Generate rollouts from the checkpoint!

After we load the checkpoint, we can do inference on the rollout!

#### Generate rollout for a single clip

The following cell will generate rollout for a single clip, specified by the clip id. The first time you call the function JAX needs to complete the `JIT` compilation, which will take around 3 minutes to execute and compile. After compilation, generates the rollout will only take about 8 seconds, since it is hardware accelerated.

In [ ]:
import jax

# first pass will take ~2m38s to run to compile
# after complied, it only takes ~9 seconds to run
output = generate_rollout(5)

#### Batch Generating Rollouts

Alternatively, you can use `jax.vmap` to parallelize the rollout function. To do so, pass a 1D array of clip indexes (`clip_idxs`) as input. 

On the first call, JAX will perform `JIT` compilation, which takes approximately **3 minutes**. Once compiled, subsequent rollouts execute in just **8 seconds**, benefiting from hardware acceleration.

**Note:** `vmap` compiles based on the input shape. This means that if you use the same length for `clip_idxs`, JAX will reuse the compiled function for acceleration. However, if the input length changes, JAX will **recompile the entire function**, incurring additional overhead.

In [12]:
# generate rollout for 5 clips simultaneously
jit_vmap_generate_rollout = jax.jit(jax.vmap(generate_rollout))
clip_idxs = jp.arange(0, 5)
jit_vmap_out = jit_vmap_generate_rollout(clip_idxs)

In [10]:
# hardware acceleration: 5 clips simultaneously
clip_idxs = jp.arange(10, 15)
jit_vmap_out2 = jit_vmap_generate_rollout(clip_idxs)

In [ ]:
# triggers recompilation: 15 clips simultaneously
clip_idxs = jp.arange(15, 30)
jit_vmap_out2 = jit_vmap_generate_rollout(clip_idxs)

### Step 4: Save it to disk

In [5]:
from track_mjx.analysis.utils import save_to_h5py, load_from_h5py
import h5py

In [ ]:
with h5py.File(
    "/root/vast/scott-yang/rodent_rollout_info/data/clip_0.h5", "w"
) as h5file:
    save_to_h5py(h5file, output)

# Render Rollout Videos from the Saved Rollouts

## Step 1: Load the rollout file from disk

Let's laodd in one rollout clip for the rodent first from our h5 file.

In [3]:
with h5py.File(
    "/root/vast/scott-yang/rodent_rollout_info/data/clip_0.h5", "r"
) as h5file:
    rollout = load_from_h5py(h5file)

## Step 2: Render rollout

Let's use the pure rendering function to render the rodent rollout first

In [6]:
frames = render_from_saved_rollout(rollout)
display_video(frames, framerate=50)

MuJoCo Rendering...


100%|██████████| 500/500 [00:02<00:00, 222.52it/s]
